In [177]:
import os
import json
import torch
import numpy as np
import pandas as pd
from pprint import pprint
from torch.utils.data import Dataset
from itertools import product
import xml.etree.ElementTree as ET
from pathlib import Path

from utils import parse_rules, parse_panels, bbox_to_xyxy, plot_example
from avr_dataset import get_stage2_dataset

In [178]:
panels_df, rules_df = get_stage2_dataset('dataset', 'train')

In [179]:
merge_row = True
panels_df_copy = panels_df.copy()

if merge_row:
    panels_df_copy['row'] = np.where(panels_df_copy['panel'] < 3, 0, 1)
    panels_df_copy['panel'] = np.where(panels_df_copy['panel'] < 3, panels_df_copy['panel'], panels_df_copy['panel'] - 3)
    reshaped_indices = ['file', 'component', 'row', 'panel']
else:
    reshaped_indices = ['file', 'component', 'panel']

reshaped_panels_df = panels_df_copy.set_index(reshaped_indices).unstack(level=-1)
reshaped_panels_df.columns.names = ['slot_attr', 'panel']
reshaped_panels_df.columns = reshaped_panels_df.columns.swaplevel(0, 1)
reshaped_panels_df = reshaped_panels_df.sort_index(axis=1, level=0)

index_tuples = []
for panel_idx, slot_idx, attr in list(product(range(0, 3) if merge_row else range(0, 6),
                                              range(0, 22),
                                              ['color', 'size', 'type'])):
    index_tuples.append((panel_idx, f'slot{slot_idx}_{attr}'))
# index_tuples = list(product(range(0, 6), range(0, 22), ['color', 'size', 'type']))
multi_index = pd.MultiIndex.from_tuples(index_tuples, names=['panel', 'slot_attr'])
reshaped_panels_df = pd.DataFrame(reshaped_panels_df, columns=multi_index)

reshaped_panels_df.columns = reshaped_panels_df.columns.map(lambda x: 'panel' + '_'.join(list(map(str, x))))

rules_df = rules_df.rename(columns={'file_path': 'file'})
rules_df = rules_df.set_index(['file', 'component'])

final_df = reshaped_panels_df.join(rules_df)

final_df

panel0_slot0_color  \
file                                               component row                       
dataset/center_single/RAVEN_0_train                0         0                    -1   
                                                             1                    -1   
dataset/center_single/RAVEN_100_train              0         0                    -1   
                                                             1                    -1   
dataset/center_single/RAVEN_101_train              0         0                    -1   
...                                                                              ...   
dataset/up_center_single_down_center_single/RAV... 1         1                    -1   
dataset/up_center_single_down_center_single/RAV... 0         0                    -1   
                                                             1                    -1   
                                                   1         0                    -1   
                                                             1                    -1   

                                                                  panel0_slot0_size  \
file                                               component row                      
dataset/center_single/RAVEN_0_train                0         0                   -1   
                                                             1                   -1   
dataset/center_single/RAVEN_100_train              0         0                   -1   
                                                             1                   -1   
dataset/center_single/RAVEN_101_train              0         0                   -1   
...                                                                             ...   
dataset/up_center_single_down_center_single/RAV... 1         1                   -1   
dataset/up_center_single_down_center_single/RAV... 0         0                   -1   
                                                             1                   -1   
                                                   1         0                   -1   
                                                             1                   -1   

                                                                  panel0_slot0_type  \
file                                               component row                      
dataset/center_single/RAVEN_0_train                0         0                   -1   
                                                             1                   -1   
dataset/center_single/RAVEN_100_train              0         0                   -1   
                                                             1                   -1   
dataset/center_single/RAVEN_101_train              0         0                   -1   
...                                                                             ...   
dataset/up_center_single_down_center_single/RAV... 1         1                   -1   
dataset/up_center_single_down_center_single/RAV... 0         0                   -1   
                                                             1                   -1   
                                                   1         0                   -1   
                                                             1                   -1   

                                                                  panel0_slot1_color  \
file                                               component row                       
dataset/center_single/RAVEN_0_train                0         0                    -1   
                                                             1                    -1   
dataset/center_single/RAVEN_100_train              0         0                    -1   
                                                             1                    -1   
dataset/center_single/RAVEN_101_train              0         0                    -1   
...                                      

In [201]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
label_columns = ['number', 'position', 'type', 'size', 'color']

X, Y = final_df[final_df.columns[:-5]], final_df[final_df.columns[-5:]]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(X)
X_train_encoded = enc.transform(X_train).toarray()
X_test_encoded = enc.transform(X_test).toarray()

In [253]:
from sklearn import svm

clf = svm.SVC()
clf.fit(X_train_encoded, y_train['type'].values)
clf.score(X_test_encoded, y_test['type'].values)

0.4866850321395776

In [249]:
label_map = {'Constant': 0, 'Distribute_Three': 1, 'Progression': 2, 'Arithmetic': 3}

In [243]:
from xgboost import XGBClassifier

bst_type = XGBClassifier(n_estimators=20, max_depth=50, learning_rate=1, objective='multi:softprob')
bst_type.fit(X_train_encoded, y_train['type'].map(label_map).values)
bst_type.score(X_test_encoded, y_test['type'].map(label_map).values)

0.8523875114784206

In [251]:
bst_color = XGBClassifier(n_estimators=20, max_depth=100, learning_rate=0.1, objective='multi:softprob')
bst_color.fit(X_train_encoded, y_train['color'].map(label_map).values)
bst_color.score(X_test_encoded, y_test['color'].map(label_map).values)

0.6779155188246098

In [252]:
bst_size = XGBClassifier(n_estimators=20, max_depth=100, learning_rate=0.1, objective='multi:softprob')
bst_size.fit(X_train_encoded, y_train['size'].map(label_map).values)
bst_size.score(X_test_encoded, y_test['size'].map(label_map).values)

0.73989898989899

In [255]:
bst_number = XGBClassifier(n_estimators=20, max_depth=100, learning_rate=0.1, objective='multi:softprob')
bst_number.fit(X_train_encoded, y_train['number'].map(label_map).values)
bst_number.score(X_test_encoded, y_test['number'].map(label_map).values)

0.8675390266299358

In [256]:
bst_position = XGBClassifier(n_estimators=20, max_depth=100, learning_rate=0.1, objective='multi:softprob')
bst_position.fit(X_train_encoded, y_train['position'].map(label_map).values)
bst_position.score(X_test_encoded, y_test['position'].map(label_map).values)

0.8675390266299358